<a href="https://colab.research.google.com/github/JimKing100/SF_Real_Estate_Project/blob/master/SF%20Map%20Code%20Test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The San Francisco Real Estate Project

In [0]:
# Install fiona
#%%capture
#!pip install fiona

In [0]:
# Install geopandas
#%%capture
#!pip install geopandas

In [0]:
# Import libraries
import pandas as pd
import numpy as np

import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

from bokeh.io import curdoc
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

### Load and Clean the Data

In [15]:
# Load SF real estate data - 10 years (2009-2018) of single family home sales in San Francisco downloaded from the SF MLS
# Longitude and latitude were added to the csv file prior to loading using geocoding.geo.census.gov 
initial_data = pd.read_csv('https://raw.githubusercontent.com/JimKing100/SF_Real_Estate_Project/master/data/SF-SFR-Sales-Final1.csv')

# Rename subdistr_desc to neighborhood
temp_data = initial_data.rename(columns={'subdist_desc': 'neighborhood'})

# Check the data
print(temp_data.shape)
temp_data.head(5)

(23711, 30)


,longitude,latitude,full_address,city,state,street_no,street_name,street_nox,street_suffix,zip,area,district_no,district_desc,subdist_no,neighborhood,on_market_data,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning
0,-122.466620,37.753273,"1878 10th Ave, San Francisco, CA 94122-4602",San Francisco,CA,1878,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,8/31/11,65,649000,12/22/11,580000,6,1.0,2,T,Per Tax Records,1480,NaN,0,1932,RH1
1,-122.466670,37.753850,"1842 10th Ave, San Francisco, CA 94122-4602",San Francisco,CA,1842,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,10/8/08,164,679000,3/23/09,509000,0,1.0,2,T,Per Tax Records,920,NaN,0,1908,RH1
2,-122.466700,37.754070,"1829 10th Ave, San Francisco, CA 94122",San Francisco,CA,1829,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,9/6/13,15,699000,10/11/13,840000,0,1.5,2,T,Per Tax Records,1200,0.0693,3017,1906,RH1
3,-122.466965,37.758057,"1612 10th Ave, San Francisco, CA 94122",San Francisco,CA,1612,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,6/8/10,9,699900,7/29/10,817000,6,1.0,3,T,Per Tax Records,1800,NaN,0,1914,NaN
4,-122.466910,37.757130,"1671 10th Ave, San Francisco, CA 94122",San Francisco,CA,1671,10th,NaN,Ave,94122,2060,2,SF District 2,2060,2 - Inner Sunset,9/8/11,25,749000,10/6/11,785000,5,1.0,2,T,Per Tax Records,1050,NaN,0,1938,RH2


In [0]:
# Fill house square foot zero values with the average house square footage by bedroom for all single family homes in SF
averagesf_data = temp_data.groupby('beds').sf.mean()

# Use average sf by bedroom for each 0 value in each bedroom group up to 9 bedrooms
for i in range(0, 9): 
  temp_data.loc[(temp_data['sf'] == 0) & (temp_data['beds'] == i), 'sf'] = averagesf_data.loc[i]

# Use 10,000sf for anything over 9 bedrooms
temp_data.loc[temp_data['sf'] == 0, 'sf'] = 10000
temp_data = temp_data.astype({'sf': int})

In [0]:
# Map MLS neighborhood code (e.g. 1010) to GeoDataFrame neighborhood id (e.g. '1a') to create the sf_data for all 10 years
neighborhood_dict = {1010: '1a', 1020: '1b', 1030: '1c', 1040: '1d', 1050: '1e', 1060: '1f', 1070: '1g',
                     2010: '2a', 2020: '2b', 2030: '2c', 2040: '2d', 2050: '2e', 2060: '2f', 2070: '2g',
                     3010: '3a', 3020: '3b', 3030: '3c', 3040: '3d', 3050: '3e', 3060: '3f', 3070: '3g', 3080: '3h', 3090: '3j',
                     4010: '4a', 4020: '4b', 4030: '4c', 4040: '4d', 4050: '4e', 4060: '4f', 4070: '4g', 4080: '4h', 4090: '4j', 4100: '4k', 4110: '4m', 4120: '4n', 4130: '4p', 4140: '4r', 4150: '4s', 4160: '4t',
                     5010: '5a', 5020: '5b', 5030: '5c', 5040: '5d', 5050: '5e', 5060: '5f', 5070: '5g', 5080: '5h', 5090: '5j', 5100: '5k', 5110: '5m',
                     6010: '6a', 6020: '6b', 6030: '6c', 6040: '6d', 6050: '6e', 6060: '6f',
                     7010: '7a', 7020: '7b', 7030: '7c', 7040: '7d', 
                     8010: '8a', 8020: '8b', 8030: '8c', 8040: '8d', 8050: '8e', 8060: '8f', 8070: '8g', 8080: '8h', 8090: '8j',
                     9010: '9a', 9020: '9c', 9030: '9d', 9040: '9e', 9050: '9f', 9060: '9h', 9070: '9j', 9080: '9g', 
                     10010: '10a', 10020: '10b', 10030: '10c', 10040: '10d', 10050: '10e', 10060: '10f', 10070: '10g', 10080: '10h', 10090: '10j', 10100: '10k', 10110: '10m', 10120: '10n'
                    }
sf_data = temp_data.replace({'subdist_no': neighborhood_dict})

In [18]:
# Create a year column based on the sale date
sf_data['year'] = '20' + sf_data['sale_date'].str[-2:]
sf_data.head(5)

# Create a price_sf column
sf_data['price_sf'] = sf_data['sale_price'] / sf_data['sf']
sf_data.head(5)

,longitude,latitude,full_address,city,state,street_no,street_name,street_nox,street_suffix,zip,area,district_no,district_desc,subdist_no,neighborhood,on_market_data,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,year,price_sf
0,-122.466620,37.753273,"1878 10th Ave, San Francisco, CA 94122-4602",San Francisco,CA,1878,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,8/31/11,65,649000,12/22/11,580000,6,1.0,2,T,Per Tax Records,1480,NaN,0,1932,RH1,2011,391.891892
1,-122.466670,37.753850,"1842 10th Ave, San Francisco, CA 94122-4602",San Francisco,CA,1842,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,10/8/08,164,679000,3/23/09,509000,0,1.0,2,T,Per Tax Records,920,NaN,0,1908,RH1,2009,553.260870
2,-122.466700,37.754070,"1829 10th Ave, San Francisco, CA 94122",San Francisco,CA,1829,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,9/6/13,15,699000,10/11/13,840000,0,1.5,2,T,Per Tax Records,1200,0.0693,3017,1906,RH1,2013,700.000000
3,-122.466965,37.758057,"1612 10th Ave, San Francisco, CA 94122",San Francisco,CA,1612,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,6/8/10,9,699900,7/29/10,817000,6,1.0,3,T,Per Tax Records,1800,NaN,0,1914,NaN,2010,453.888889
4,-122.466910,37.757130,"1671 10th Ave, San Francisco, CA 94122",San Francisco,CA,1671,10th,NaN,Ave,94122,2060,2,SF District 2,2f,2 - Inner Sunset,9/8/11,25,749000,10/6/11,785000,5,1.0,2,T,Per Tax Records,1050,NaN,0,1938,RH2,2011,747.619048


In [19]:
# Create summary data by neighborhood
neighborhood_data = sf_data.groupby(['year', 'subdist_no'])['price_sf'].mean()

# Reset the index to 1 level to fill in year
neighborhood_data = neighborhood_data.reset_index(level=[0,1])

# Change data types to integer for price_sf and year
neighborhood_data = neighborhood_data.astype({'price_sf': 'int'})
neighborhood_data = neighborhood_data.astype({'year': 'int'})
neighborhood_data.head()

,year,subdist_no,price_sf
0,2009,10a,287
1,2009,10b,420
2,2009,10c,440
3,2009,10d,438
4,2009,10e,355


### Prepare the mapping data and GeoDataFrame

In [20]:
# Read the geojon map file for Realtor Neighborhoods into a GeoDataframe object
sf = geopandas.read_file('https://raw.githubusercontent.com/JimKing100/SF_Real_Estate_Project/master/data/Realtor%20Neighborhoods.geojson')

# Set the Coordinate Referance System (crs) or projections
# ESPG code 4326 is also referred to as WGS84 lat-long projection
sf.crs = {'init': 'epsg:4326'}

# Rename columns in geojson map file
sf = sf.rename(columns={'geometry': 'geometry','nbrhood':'neighborhood_name', 'nid': 'subdist_no'}).set_geometry('geometry')

# Change neighborhood id (subdist_no) for correct code for Mount Davidson Manor and for parks
sf.loc[sf['neighborhood_name'] == 'Mount Davidson Manor', 'subdist_no'] = '4n'
sf.loc[sf['neighborhood_name'] == 'Golden Gate Park', 'subdist_no'] = '12a'
sf.loc[sf['neighborhood_name'] == 'Presidio', 'subdist_no'] = '12b'
sf.loc[sf['neighborhood_name'] == 'Lincoln Park', 'subdist_no'] = '12c'

sf.sort_values(by=['subdist_no'])
sf.head()

,neighborhood_name,subdist_no,sfar_distr,geometry
0,Alamo Square,6e,District 6 - Central North,(POLYGON ((-122.4294839489174 37.7750962307043...
1,Anza Vista,6a,District 6 - Central North,(POLYGON ((-122.4474643913587 37.7798633530923...
2,Balboa Terrace,4a,District 4 - Twin Peaks West,(POLYGON ((-122.464508862148 37.73220849554402...
3,Bayview,10a,District 10 - Southeast,"(POLYGON ((-122.38758527039 37.7502633777501, ..."
4,Bernal Heights,9a,District 9 - Central East,(POLYGON ((-122.4037549223623 37.7491900637356...


In [0]:
# Pull the intial 2018 data from neighborhood summary data
#df_2018 = neighborhood_data[neighborhood_data['year'] == 2018]

# Merge the GeoDataframe object (sf) with the neighborhood summary data (neighborhood)
#merged = pd.merge(sf, df_2018, on='subdist_no', how='left')
    
# Fill the null values with in year and price_sf
#values = {'year': 2018, 'price_sf': 0}
#merged = merged.fillna(value=values)
    
# Bokeh uses geojson formatting, representing geographical features, with json
# Convert to json
#merged_json = json.loads(merged.to_json())

# Convert to json preferred string-like object 
#json_data = json.dumps(merged_json)

# Input geojson source that contains features for plotting for the initial year 2018
#geosource = GeoJSONDataSource(geojson = json_data)

### Create the Interactive Plot

In [0]:
# Create a function the returns json_data for the year selected by the user
def json_data(selectedYear):
    yr = selectedYear
    
    # Pull selected year from neighborhood summary data
    df_yr = neighborhood_data[neighborhood_data['year'] == yr]
    
    # Merge the GeoDataframe object (sf) with the neighborhood summary data (neighborhood)
    merged = pd.merge(sf, df_yr, on='subdist_no', how='left')
    
    # Fill the null values with in year and price_sf
    values = {'year': yr, 'price_sf': 0}
    merged = merged.fillna(value=values)
    
    # Bokeh uses geojson formatting, representing geographical features, with json
    # Convert to json
    merged_json = json.loads(merged.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data
  
# Define the callback function: update_plot
def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data(yr)
    geosource.geojson = new_data
    #p.patches('xs','ys', source = geosource, fill_color = {'field' :'price_sf', 'transform' : color_mapper},
    #           line_color = 'black', line_width = 0.25, fill_alpha = 1)
    #p.title.text = 'Price Per Square Foot by Neighborhood for Single Family Homes in SF by Year' %yr

# Input geojson source that contains features for plotting for the initial year 2018
geosource = GeoJSONDataSource(geojson = json_data(2018))

# Define a sequential multi-hue color palette.
palette = brewer['Blues'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 3000)

# Add hover tool
hover = HoverTool(tooltips = [ ('Neighborhood','@neighborhood_name'),('Price/SF ', '$@price_sf')])

# Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')

# Create figure object.
p = figure(title = 'Price Per Square Foot by Neighborhood for Single Family Homes in SF by Year', 
           plot_height = 850, plot_width = 950,
           toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.visible = False

# Add patch renderer to figure. 
p.patches('xs','ys', source = geosource, fill_color = {'field' :'price_sf', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

# Specify figure layout.
p.add_layout(color_bar, 'below')

# Add the hover tool to the graph
p.add_tools(hover)
    
# Make a slider object: slider 
slider = Slider(title = 'Year', start = 2009, end = 2018, step = 1, value = 2018)
slider.on_change('value', update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p, widgetbox(slider))
curdoc().add_root(layout)

# Display figure inline in Notebook.
#output_file('test2.html')
#output_notebook()
